<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Stripe - Send payments status by email

**Tags:** #stipe #finance #naas #naas_drivers #snippet #dataframe #notifications

**Author:** [Jeremy Ravenel](https://www.linkedin.com/in/jeremyravenel/)

**Last update:** 2023-08-15

**Description:** This notebook provides a way to retrieve payments from Stripe API and generate an AI Plugin for Naas Chat. 

## Input

### Import libraries

In [ ]:
!pip install stripe
!pip install pandasql

In [ ]:
import os
import json
import naas 
import naas_drivers
import stripe
import plotly.graph_objects as go
import requests
from naas_drivers import emailbuilder
from datetime import datetime, date
import random
import time
from dateutil.parser import parse
import matplotlib.pyplot as plt
import pandas as pd

try:
    import tiktoken
except:
    !pip install tiktoken --user
    import tiktoken

### Setup variables

In [ ]:
# Define scenario 
is_demo = True # Default to True

# For Stripe API 
# Set your secret key. Remember to switch to your live secret key in production!
# See your keys here: https://dashboard.stripe.com/account/apikeys
stripe_api = naas.secret.get("STRIPE_API")

# For Emails
EMAIL_TO = "jeremy@naas.ai"  # you will receive weekly summary at this email
EMAIL_FROM = None  # summary will have this email as sender. Only available for your naas email, otherwise you will receive this email from notification@naas.ai
EMAIL_SUBJECT = (f"💰 Finance Management Demo - Email Update, {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
DATE_FORMAT = "%Y-%m-%d"


# For openAI API
openai_api_key = naas.secret.get("OPENAI_API_KEY")


# For AI Chat Plugin
plugin_name = "💰 Finance Assistant"
plugin_model = "gpt-3.5-turbo"
plugin_temperature = 0
plugin_max_tokens = 8192
system_prompt_max_tokens = 2084


# For Asset Generation
output_dir = "/home/ftp/abi/outputs/by_modules/financial_transactions"
csv_file_name = "data.csv"
image_file_name = "image.png"
plugin_file_name = "plugin.json"

## Model

### Setup directories

In [ ]:
# Check if directory exists and create it if not
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# Generate outputs files path
csv_file_path = os.path.join(output_dir, csv_file_name)
image_file_path = os.path.join(output_dir, image_file_name)
plugin_file_path = os.path.join(output_dir, plugin_file_name)
print('📂 CSV file path:', csv_file_path)
print('📂 Image file path:', image_file_path)
print('📂 Plugin file path:', plugin_file_path)

### Get or load data

In [ ]:
def get_or_load_data(api_key, demo_mode=is_demo):
    # Set the Stripe API key
    stripe.api_key = api_key
    
    # Init
    df = pd.DataFrame()
    update_data = True
    
    # Check if demo_mode is True
    if demo_mode:
        file_path = "/home/ftp/abi/inputs/demo_data/demo_data-stripe_payments.csv"
        if os.path.exists(file_path):
            # Read file
            df = pd.read_csv(file_path)
            df.to_csv(csv_file_path, index=False)
            return df
        
    # Check if the file exists
    if os.path.isfile(csv_file_path):
        # If the file exists, load it into a DataFrame
        return pd.read_csv(csv_file_path)
    else:
        # Get a list of charges
        charges = stripe.Charge.list()

        # Prepare a list to store the charge data
        data = []

        # Loop through the charges and store the data
        for charge in charges.auto_paging_iter():
            # Retrieve the customer object
            customer = stripe.Customer.retrieve(charge.customer)
            # Retrieve the payment method object
            payment_method = stripe.PaymentMethod.retrieve(charge.payment_method)

            # Check if the charge has an associated invoice
            if charge.invoice:
                # Retrieve the invoice object to get line items
                invoice = stripe.Invoice.retrieve(charge.invoice)
                # Retrieve the first line item's price object
                price = invoice.lines.data[0].price if invoice.lines.data else None
                # Retrieve the product object using the product id from the price object
                product = stripe.Product.retrieve(price.product) if price else None
            else:
                product = None

            data.append({
                'id': charge.id,
                'date': pd.to_datetime(charge.created, unit='s'),  # Stripe's timestamps are in seconds
                'amount': charge.amount / 100.0,  # Stripe's amounts are in cents, so we divide by 100
                'currency': charge.currency.upper(),
                'description': charge.description,
                'customer': customer.name if customer.name else customer.email,  # Use name if available, else use email
                'product': product.name if product else None,  # Get the product name
                'status': charge.status,
                'payment_method': payment_method.type,
            })
        # Convert the dictionary to a DataFrame and save it to a CSV file
        df = pd.DataFrame.from_dict(contrib_dict, orient='index')
        return df
    
df = get_or_load_data(stripe_api)
df

### Prepare data for chart

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['YMD'] = df['date'].dt.strftime('%Y-%m-%d')
df['HMS'] = df['date'].dt.strftime('%H:%M:%S')
df


In [ ]:
from pandasql import sqldf

def aggregate_by_date_and_amount(df):
    # Define the SQL query
    query = """
        SELECT
            SUBSTR(YMD, 1, 7) as year_month,
            ROUND(SUM(amount), 2) as total_amount
        FROM
            df
        GROUP BY
            year_month
    """

    # Execute the SQL query and return the result
    return sqldf(query)

df_aggregated = aggregate_by_date_and_amount(df)
df_aggregated

### Create Chart

In [ ]:
import plotly.graph_objects as go

def create_vertical_barchart(df_aggregated, label="year_month", value="total_amount"):
    # Convert 'year_month' to string format
    df_aggregated[label] = df_aggregated[label].astype(str)

    # Filter out rows where the value is zero
    df_non_zero = df_aggregated[df_aggregated[value] != 0]

    # Init
    fig = go.Figure()

    # Create fig
    fig.add_trace(
        go.Bar(
            x=df_non_zero[label],   # This represents the categories/labels
            y=df_non_zero[value],   # This represents the values
            text=df_non_zero[value],  # Display the value at the end of the bar
            textposition="outside",
            marker=dict(color="#6259fa"),
            orientation="v"  # Set to "h" for horizontal
        )
    )

    fig.update_traces(showlegend=False)

    # Update x-axis tick labels to show dates in format YYYY-MM
    fig.update_xaxes(
        tickvals=df_non_zero[label],  # Use the actual date values for tick positions
        ticktext=df_non_zero[label].str[:7],  # Extract YYYY-MM from the date strings
        type='category'  # Set the x-axis type to 'category'
    )

    # Plotly: Create title
    title = f"<b><span style='font-size: 20px;'>Stripe payments cashed-in over the past months</b>"
    fig.update_layout(
        title=title,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
    )

    return fig

# You can call the function with a dataframe
create_vertical_barchart(df_aggregated)


### Create asset from chart

In [ ]:
import plotly.io as pio

# Create the chart
chart = create_vertical_barchart(df_aggregated)

# Save as PNG
pio.write_image(chart, image_file_path)

#graph_url = naas.asset.add("chart.html", {"inline": True})
stripe_chart = naas.asset.add(image_file_path)

### Set email parameters

In [ ]:
today = datetime.now().strftime(DATE_FORMAT)
today

### Create NaasAI Chat plugin

In [ ]:
system_prompt = f"""
Act as a Financial Management Agent with access to comprehensive financial data sources, including detailed transaction records from Stripe API. 
Your primary objective is to analyze and optimize financial transactions, ensuring you identify key insights and trends to guide financial strategies. 
Leverage the data to decipher patterns, customer behavior, and payment methods to strategize on revenue growth and financial operations.
Your ultimate goal is to maximize revenue, minimize risks, and contribute to the overall financial success of the organization.
- Start by introducing yourself with a maximum of 5 bullet points.
- Display the current financial analytics data as an image inside the markdown of the chat: {stripe_chart}.
Wait for the user's initial response, and then delve into a high-level analysis of the financial transactions data.
Here is the specific financial analytics data you should emphasize: {df_aggregated}
"""

### Check token count 

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

system_prompt_tokens = num_tokens_from_string(system_prompt, "cl100k_base")
if system_prompt_tokens > system_prompt_max_tokens:
    print("⚠️ Be carefull, your system prompt looks too big. Tokens:", system_prompt_tokens)
else:
    print("✅ System prompt tokens count OK:", system_prompt_tokens)

### Generate Plugin

In [ ]:
# Create json
plugin = {
    "name": plugin_name,
    "model": plugin_model,
    "temperature": plugin_temperature,
    "max_tokens": plugin_max_tokens,
    "prompt": system_prompt,
}

# Save dict to JSON file
with open(plugin_file_path, "w") as f:
    json.dump(plugin, f)
print("💾 Plugin successfully saved:")

plugin = naas.asset.add(plugin_file_path, params={"inline": True})

### Create Top 10 Customers

In [ ]:
from pandasql import sqldf
import pandas as pd

def aggregate_by_customer(df):
    # Define the SQL query
    query = """
        SELECT
            customer,
            CAST(SUM(amount) AS INTEGER) as total_amount
        FROM
            df
        GROUP BY
            customer
        ORDER BY
            total_amount DESC
    """
    # Execute the SQL query and return the result
    result_df = sqldf(query)
    
    return result_df

df_aggregated_by_customer = aggregate_by_customer(df)
df_aggregated_by_customer

In [ ]:
def format_number(num):
    NUMBER_FORMAT = "{:,.0f}"
    num = str(NUMBER_FORMAT.format(num)).replace(",", " ")
    return num

In [ ]:
import pandas as pd

def top10_clients(df, top_n=10):
    # Group the data by customer and calculate total amounts
    grouped = df.groupby('customer')['amount'].sum().reset_index()
    
    # Get top 10 clients based on total amounts
    top_clients = grouped.nlargest(top_n, 'amount')
    
    # Format the top clients for email incorporation
    email_list = []
    for idx, (_, row) in enumerate(top_clients.iterrows(), 1):
        client_name = row['customer']
        total_amount = row['amount']
        email_list.append(f"{idx}. {client_name} : <b>${total_amount}</b>")
    
    return email_list

# Use the function
top_clients_list = top10_clients(df)
print("\n".join(top_clients_list))

## Output

### Create email content

In [ ]:
def email_brief(
    today,
    top_clients_list,

):
    content = {
        'title': ("💰 Finance Engine - Email Update"),
        'heading': (f"Date:{today}"),
        "txt_intro": (
            f"Hi there,<br><br>" f"Here is your finance engine email as of {today}."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="27px", text_align="center", bold=True
        ),
        "text_1": emailbuilder.text(
            f"Here are the total payments history from Stripe:"
        ),
        "image_1": emailbuilder.image(stripe_chart),
        "title_2": emailbuilder.text(
            "Top 10", font_size="27px", text_align="center", bold=True
        ),
        "text": emailbuilder.list(top_clients_list),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin}"),
            text="Start Chatting With Agent",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    today,
    top_clients_list,
)

## Output

### Send post engagement by email

In [ ]:
# sends the email
naas.notification.send(
    email_to=EMAIL_TO, subject=EMAIL_SUBJECT, html=email_content, email_from=EMAIL_FROM
)